In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import open_clip
open_clip.list_pretrained()
import torch
from PIL import Image

In [17]:
model, train_preprocess, preprocess = open_clip.create_model_and_transforms('ViT-B-16', pretrained='datacomp_xl_s13b_b90k')
model.eval()  # model in train mode by default, impacts some models with BatchNorm or stochastic depth active
tokenizer = open_clip.get_tokenizer('ViT-B-16')

output_tokens:False


In [18]:
train_preprocess

Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.9, 1.0), ratio=(0.75, 1.3333), interpolation=bicubic, antialias=True)
    <function _convert_to_rgb at 0x77b22f8dfa30>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [3]:
preprocess

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    <function _convert_to_rgb at 0x77b22f8dfa30>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [4]:
model

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (patch_dropout): Identity()
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0-11): 12 x ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ls_2): Identity()
        )
      )
    )
    (ln_post): LayerNorm((768,), eps=1e-05, elementwise_affine

In [5]:
model.visual.proj.shape

torch.Size([768, 512])

In [6]:
model.visual.pool_type

'tok'

In [7]:
print(model.visual.attn_pool)

None


In [8]:
model.visual.final_ln_after_pool

False

In [9]:
model.visual.output_tokens = True

In [14]:
model.text_pool_type

'argmax'

In [16]:
model.text_projection.shape

torch.Size([512, 512])

In [10]:
image = preprocess(Image.open("/home/user/data/test_cap.png")).unsqueeze(0)
text = tokenizer(["a diagram", "a dog", "a young girl", "a cat"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features, token_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

Label probs: tensor([[2.9387e-05, 8.0333e-05, 9.9950e-01, 3.8842e-04]])


每张图片分为196+1个patch（$(224/16)^2+1$），1个patch为cls token，image_features是取的这个cls token，其余的可通过token_features返回。

In [11]:
token_features.shape

torch.Size([1, 196, 768])

In [12]:
image_features.shape

torch.Size([1, 512])

In [13]:
text_features.shape

torch.Size([4, 512])